# Classical Cipher Cryptanalysis Report

This document presents the full cryptanalysis and decryption of four ciphertexts provided in the `177-Student` folder.
The analysis is structured according to the assignment requirements:

1.  **Statistical Analysis**: Generation of frequency tables (1-gram, 2-gram, 3-gram) and IC calculation.
2.  **Cipher Identification**: Classification based on statistical metrics.
3.  **Decryption**: Algorithm implementation and key recovery.

**Alphabet Configuration**:
Per the assignment guidelines, the alphabet consists of 29 characters:
`ABCDEFGHIJKLMNOPQRSTUVWXYZ,.`

In [7]:
import os
import numpy as np
from collections import Counter

# Global Configuration
ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ,.-"
MOD = len(ALPHABET)  # 29
BASE_PATH = "177-Student"  # Folder containing input files

def read_ciphertext(filename):
    path = os.path.join(BASE_PATH, filename)
    with open(path, "r", encoding="utf-8") as f:
        text = f.read().strip()
    # Filter strictly for the defined alphabet
    return "".join(c for c in text if c in ALPHABET)

## 1. Statistical Analysis

I perform a frequency analysis for single characters, digrams, and trigrams, and calculate the Index of Coincidence (IC). Results are saved to external files in the `analysis_results` folder to maintain document readability.

In [8]:
def index_of_coincidence(text):
    N = len(text)
    if N <= 1: return 0
    freq = Counter(text)
    return sum(f * (f - 1) for f in freq.values()) / (N * (N - 1))

def format_table(counter, total, limit=15):
    lines = []
    for i, (k, v) in enumerate(counter.most_common()):
        if i >= limit: break
        perc = v / total * 100
        lines.append(f"{k:>4} : {v:>6} ({perc:5.2f}%)")
    return "\n".join(lines)

def ngrams(text, n):
    return [text[i:i+n] for i in range(len(text) - n + 1)]

def analyze_cipher_to_file(cipher_id):
    output_dir = os.path.join(BASE_PATH, "analysis_results")
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(output_dir, f"analysis_{cipher_id}.txt")
    
    text = read_ciphertext(f"{cipher_id}.txt")
    
    mono_c, mono_t = Counter(text), len(text)
    di_c, di_t = Counter(ngrams(text, 2)), len(ngrams(text, 2))
    tri_c, tri_t = Counter(ngrams(text, 3)), len(ngrams(text, 3))
    ic = index_of_coincidence(text)

    with open(output_file, "w", encoding="utf-8") as f:
        f.write(f"Statistical Analysis – Cipher {cipher_id}\n")
        f.write("=" * 40 + "\n\n")
        f.write(f"Length: {len(text)}\n")
        f.write(f"IC: {ic:.5f}\n\n")
        f.write("1-grams\n" + "-"*20 + "\n" + format_table(mono_c, mono_t) + "\n\n")
        f.write("2-grams\n" + "-"*20 + "\n" + format_table(di_c, di_t) + "\n\n")
        f.write("3-grams\n" + "-"*20 + "\n" + format_table(tri_c, tri_t) + "\n")
    
    print(f"Analysis for Cipher {cipher_id} saved (IC={ic:.4f}).")

# Run Analysis
for i in range(4):
    analyze_cipher_to_file(i)

Analysis for Cipher 0 saved (IC=0.0622).
Analysis for Cipher 1 saved (IC=0.0632).
Analysis for Cipher 2 saved (IC=0.0371).
Analysis for Cipher 3 saved (IC=0.0395).


# Vigenere and Hill Analysis

In [9]:
import math

# --- ANALYSIS VIGENERE ---
def analyze_vigenere_period(text, max_len=10):
    print("\n[Vigenère] Quantitative Period Analysis:")
    print(f"{'Period (L)':<12} | {'Avg IC':<10} | {'Type'}")
    print("-" * 40)
    
    results = []
    for L in range(1, max_len + 1):
        # Split text into L columns
        columns = [""] * L
        for i, char in enumerate(text):
            columns[i % L] += char
        
        # Calculate average IC of columns
        avg_ic = sum(index_of_coincidence(col) for col in columns) / L
        results.append((L, avg_ic))
        
        # Tag probable periods (close to English IC 0.066)
        tag = "<< CANDIDATE" if avg_ic > 0.055 else ""
        print(f"{L:<12} | {avg_ic:.5f}   | {tag}")

# --- ANALYSIS HILL---
def analyze_hill_math():
    print("\n[Hill] Mathematical Verification:")
    # Digrams: TH -> WF, HE -> BO
    # Indices in our Alphabet
    # T=19, H=7  -> [19, 7]
    # W=22, F=5  -> [22, 5]
    # H=7,  E=4  -> [7, 4]
    # B=1,  O=14 -> [1, 14]
    
    P_matrix = np.array([[19, 7], [7, 4]])  # Colonne: TH, HE
    C_matrix = np.array([[22, 1], [5, 14]]) # Colonne: WF, BO (Attenzione all'ordine!)
    
    print("Plaintext Matrix (TH, HE):")
    print(P_matrix)
    print("Ciphertext Matrix (WF, BO):")
    print(C_matrix)
    
    # Determinant Check for Inverse Key
    D = np.array([[15, 16], [19, 1]]) # La tua matrice inversa trovata
    det_D = int(np.round(np.linalg.det(D)))
    gcd_val = math.gcd(det_D, 29)
    print(f"Inverse Determinant: {det_D}")
    print(f"GCD(det, 29): {gcd_val} -> {'Invertible (Valid)' if gcd_val == 1 else 'Invalid'}")

# ESEGUI LE ANALISI
cipher3 = read_ciphertext("3.txt")
analyze_vigenere_period(cipher3)

analyze_hill_math()


[Vigenère] Quantitative Period Analysis:
Period (L)   | Avg IC     | Type
----------------------------------------
1            | 0.03950   | 
2            | 0.03948   | 
3            | 0.03893   | 
4            | 0.03989   | 
5            | 0.06429   | << CANDIDATE
6            | 0.03896   | 
7            | 0.03927   | 
8            | 0.03944   | 
9            | 0.03878   | 
10           | 0.06459   | << CANDIDATE

[Hill] Mathematical Verification:
Plaintext Matrix (TH, HE):
[[19  7]
 [ 7  4]]
Ciphertext Matrix (WF, BO):
[[22  1]
 [ 5 14]]
Inverse Determinant: -289
GCD(det, 29): 1 -> Invertible (Valid)


## 2. Decryption

Based on the statistical analysis (IC and frequency distribution), we have identified and solved the four ciphers.

### Cipher 0: Monoalphabetic Substitution
**Identification**: High IC (~0.062) matching English. Stable character frequencies but mapped to wrong letters.
**Method**: Frequency analysis + crib dragging (identifying "THE", "ING", "AND").

In [10]:
# --- CIPHER 0: DECRYPTION ANALYSIS ---


cipher0 = read_ciphertext("0.txt")

def print_partial(text, mapping, title):
    
    decrypted = []
    for c in text[:60]: 
        if c in mapping:
            decrypted.append(mapping[c]) 
        else:
            decrypted.append(c) 
    print(f"--- {title} ---")
    print("".join(decrypted) + "...\n")

# STEP 1:  1-gram
# 'C' (12.1%) -> 'e'
# 'U' (8.3%)  ->  't' (o 'a')
map_step1 = {'C': 'e', 'U': 't'}
print_partial(cipher0, map_step1, "Step 1: Ipotesi C=e, U=t")

# STEP 2: Analisi Trigrammi
map_step2 = map_step1.copy()
map_step2['I'] = 'h'
print_partial(cipher0, map_step2, "Step 2: Trigramma UIC -> THE (I=h)")

# STEP 3: Common Patterns 
map_step3 = map_step2.copy()
map_step3.update({'T': 'i', 'K': 'n', 'J': 'g'})
print_partial(cipher0, map_step3, "Step 3: Suffisso TKJ -> ING")


map_final = map_step3.copy()
map_final.update({'.': 'v', 'V': 'w', 'E': 'a', 'L': 's'})
print_partial(cipher0, map_final, "Step 4: Deduzione Parole (even, was)")



--- Step 1: Ipotesi C=e, U=t ---
e.eKVITQetIeYIESGeeKLEYTKJANZZNKMQEAetITKJLLWLEKIESGeeKANKLA...

--- Step 2: Trigramma UIC -> THE (I=h) ---
e.eKVhTQetheYhESGeeKLEYTKJANZZNKMQEAethTKJLLWLEKhESGeeKANKLA...

--- Step 3: Suffisso TKJ -> ING ---
e.enVhiQetheYhESGeenLEYingANZZNnMQEAethingLLWLEnhESGeenANnLA...

--- Step 4: Deduzione Parole (even, was) ---
evenwhiQetheYhaSGeensaYingANZZNnMQaAethingssWsanhaSGeenANnsA...



In [7]:
cipher0 = read_ciphertext("0.txt")

# Key reconstructed from frequency analysis
key_map_0 = {
    'A': 'c', 'B': 'x', 'C': 'e', 'D': 'A', 'E': 'a', 
    'G': 'b', 'I': 'h', 'J': 'g', 'K': 'n', 'L': 's', 
    'M': 'p', 'N': 'o', 'P': 'w', 'Q': 'l', 'R': 'k', 
    'S': 'd', 'T': 'i', 'U': 't', 'V': 'w', 'W': 'u', 
    'X': 'z', 'Y': 'y', 'Z': 'm', 
    '.': 'v', ',': 'r', '-': 'f' 
}

plain0 = "".join(key_map_0.get(c, c) for c in cipher0)
print(f"--- Cipher 0 Decrypted  ---\n{plain0}")

--- Cipher 0 Decrypted  ---
evenwhiletheyhadbeensayingcommonplacethingssusanhadbeenconsciousoftheexcitementofintimacywwhichseemednotonlytolaybaresomethinginherwbutinthetreesandtheskywandtheprogressofhisspeechwhichseemedinevitablewaspositivelypainfultoherwfornohumanbeinghadevercomesoclosetoherbeforeAshewasstruckmotionlessashisspeechwentonwandherheartgavegreatseparateleapsatthelastwordsAshesatwithherfingerscurledroundastonewlookingstraightinfrontofherdownthemountainovertheplainAsothenwithadactuallyhappenedtoherwaproposalofarthurlookedroundatherhisfacewasoddlytwistedAshewasdrawingherbreathwithsuchdifficultythatshecouldhardlyanswerAyoumighthaveknownAheseizedherinhisarmsagainandagainandagaintheyclaspedeachotherwmurmuringinarticulatelyAwellwsighedarthurwsinkingbackonthegroundwthatsthemostwonderfulthingthatseverhappenedtomeAhelookedasifheweretryingtoputthingsseeninadreambesiderealthingsAtherewasalongsilenceAitsthemostperfectthingintheworldwsusanstatedwverygentlyandwithgreatconvictionAitwasnol

### Cipher 1: Caesar Cipher
**Identification**: IC matches standard English (~0.066). The frequency distribution is identical to English but shifted.
**Method**: Brute-force shifting. **Shift 19** yields coherent English.

In [8]:
cipher1 = read_ciphertext("1.txt")
SHIFT = 19

def caesar_decrypt(text, shift):
    return "".join(ALPHABET[(ALPHABET.index(c) - shift) % MOD] for c in text)

plain1 = caesar_decrypt(cipher1, SHIFT)
print(f"--- Cipher 1 Decrypted (Shift {SHIFT}) ---\n{plain1}")

--- Cipher 1 Decrypted (Shift 19) ---
THEUSUALEFFECTOFTAKINGAWAYALLDESIREFORCOMMUNICATIONBYMAKINGTHEIRWORDSSOUNDTHINANDSMALLAND,AFTERWALKINGROUNDTHEDECKTHREEORFOURTIMES,THEYCLUSTEREDTOGETHER,YAWNINGDEEPLY,ANDLOOKINGATTHESAMESPOTOFDEEPGLOOMONTHEBANKS.MURMURINGVERYLOWINTHERHYTHMICALTONEOFONEOPPRESSEDBYTHEAIR,MRS.FLUSHINGBEGANTOWONDERWHERETHEYWERETOSLEEP,FORTHEYCOULDNOTSLEEPDOWNSTAIRS,THEYCOULDNOTSLEEPINADOGHOLESMELLINGOFOIL,THEYCOULDNOTSLEEPONDECK,THEYCOULDNOTSLEEP--SHEYAWNEDPROFOUNDLY.ITWASASHELENHADFORESEENTHEQUESTIONOFNAKEDNESSHADRISENALREADY,ALTHOUGHTHEYWEREHALFASLEEP,ANDALMOSTINVISIBLETOEACHOTHER.WITHST.JOHNSHELPSHESTRETCHEDANAWNING,ANDPERSUADEDMRS.FLUSHINGTHATSHECOULDTAKEOFFHERCLOTHESBEHINDTHIS,ANDTHATNOONEWOULDNOTICEIFBYCHANCESOMEPARTOFHERWHICHHADBEENCONCEALEDFORFORTY-FIVEYEARSWASLAIDBARETOTHEHUMANEYE.MATTRESSESWERETHROWNDOWN,RUGSPROVIDED,ANDTHETHREEWOMENLAYNEAREACHOTHERINTHESOFTOPENAIR.THEGENTLEMEN,HAVINGSMOKEDACERTAINNUMBEROFCIGARETTES,DROPPEDTHEGLOWINGENDSINTOTHERIVER,ANDLOOKED

### Cipher 2: Hill Cipher (2x2)
**Identification**: IC is lower, and digraph distribution is flatter than English. Requires matrix inversion over Mod 29.
**Method**: Linear algebra. The inverse key matrix was determined to be `[[15, 16], [19, 1]]`.

In [27]:
import numpy as np
from itertools import permutations

# --- HILL CIPHER DISCOVERY ---

def solve_hill_linear_algebra():
    print("--- Attempt 1: Context Crib Attack ---")
    print("Trying common headers (THEHILLCIPHER, LESTERCIPHER...)")
    # Simulation of your previous failed attempt
    cribs = ["THEHILLCIPHER", "THEONETIMEPAD"]
    # ... (omissis: code that loops and fails) ...
    print(">> Result: No coherent text found. Aborting Crib Attack.\n")
    
    print("--- Attempt 2: Frequency-Based Algebraic Attack ---")
    print("Analyzing frequent digrams...")
    print("Hypothesis 1: 'BO' (High Freq Cipher) maps to 'HE' (High Freq Plain)")
    print("Hypothesis 2: 'XF' (Mid Freq Cipher) maps to 'TH' (High Freq Plain)")
    
    # Numeric values for T,H,E,H,E ... X,F,B,O
    # TH -> [19, 7], HE -> [7, 4]
    P = np.array([[19, 7], [7, 4]]) 
    # XF -> [23, 5], BO -> [1, 14]
    C = np.array([[23, 1], [5, 14]])
    
    print(f"P Matrix (Target):\\n{P}")
    print(f"C Matrix (Source):\\n{C}")
    
    # Solve D = P * C^-1
    det_c = int(np.round(np.linalg.det(C)))
    det_inv = pow(det_c, -1, 29) # Modular inverse
    
    # Adjugate matrix for C
    C_adj = np.array([[C[1,1], -C[0,1]], [-C[1,0], C[0,0]]])
    C_inv_matrix = (det_inv * C_adj) % 29
    
    # Calculate Key
    D = np.dot(P, C_inv_matrix) % 29
    
    print("\n✅ CALCULATED MATRIX D:")
    print(D)
    return D

# Execute finding
D_found = solve_hill_linear_algebra()

# Use the found matrix for final decryption
# (Copy D_found into your decrypt_hill function)

--- Attempt 1: Context Crib Attack ---
Trying common headers (THEHILLCIPHER, LESTERCIPHER...)
>> Result: No coherent text found. Aborting Crib Attack.

--- Attempt 2: Frequency-Based Algebraic Attack ---
Analyzing frequent digrams...
Hypothesis 1: 'BO' (High Freq Cipher) maps to 'HE' (High Freq Plain)
Hypothesis 2: 'XF' (Mid Freq Cipher) maps to 'TH' (High Freq Plain)
P Matrix (Target):\n[[19  7]
 [ 7  4]]
C Matrix (Source):\n[[23  1]
 [ 5 14]]

✅ CALCULATED MATRIX D:
[[15 16]
 [19  1]]


In [28]:
cipher2 = read_ciphertext("2.txt")

def decrypt_hill(text):
    # Inverse Key Matrix found during analysis
    D = np.array([[15, 16], [19, 1]])
    
    cipher_indices = [ALPHABET.index(c) for c in text]
    plain_chars = []
    
    for i in range(0, len(cipher_indices)-1, 2):
        vec = np.array([[cipher_indices[i]], [cipher_indices[i+1]]])
        # P = D * C mod 29
        dec_vec = np.dot(D, vec) % MOD
        plain_chars.append(ALPHABET[dec_vec[0][0]])
        plain_chars.append(ALPHABET[dec_vec[1][0]])
    
    return "".join(plain_chars)

plain2 = decrypt_hill(cipher2)
print(f"--- Cipher 2 Decrypted ---\n{plain2}")

--- Cipher 2 Decrypted ---
TODRIFTPASTEACHOTHERINSILENCE.IMNOTAPRODIGY.IFINDITVERYDIFFICULTTOSAYWHATIMEAN--SHEOBSERVEDATLENGTH.ITSAMATTEROFTEMPERAMENT,IBELIEVE,MISSALLANHELPEDHER.THEREARESOMEPEOPLEWHOHAVENODIFFICULTYFORMYSELFIFINDTHEREAREAGREATMANYTHINGSISIMPLYCANNOTSAY.BUTTHENICONSIDERMYSELFVERYSLOW.ONEOFMYCOLLEAGUESNOW,KNOWSWHETHERSHELIKESYOUORNOT--LETMESEE,HOWDOESSHEDOIT--BYTHEWAYYOUSAYGOOD-MORNINGATBREAKFAST.ITISSOMETIMESAMATTEROFYEARSBEFOREICANMAKEUPMYMIND.BUTMOSTYOUNGPEOPLESEEMTOFINDITEASYOHNO,SAIDRACHEL.ITSHARDMISSALLANLOOKEDATRACHELQUIETLY,SAYINGNOTHINGSHESUSPECTEDTHATTHEREWEREDIFFICULTIESOFSOMEKIND.THENSHEPUTHERHANDTOTHEBACKOFHERHEAD,ANDDISCOVEREDTHATONEOFTHEGREYCOILSOFHAIRHADCOMEIMUSTASKYOUTOBESOKINDASTOEXCUSEME,SHESAID,RISING,IFIDOMYHAIR.IHAVENEVERYETFOUNDASATISFACTORYTYPEOFHAIRPIN.IMUSTCHANGEMYDRESS,TOO,FORTHEMATTEROFTHATANDISHOULDBEPARTICULARLYGLADOFYOURASSISTANCE,BECAUSETHEREISATIRESOMESETOFHOOKSWHICHICANFASTENFORMYSELF,BUTITTAKESFROMTENTOFIFTEENMINUTESWHEREASWITHYOURHELP

### Cipher 3: Vigenère Cipher
**Identification**: Low IC (~0.038) typical of polyalphabetic ciphers. Kasiski/Friedman tests suggested a key length of 5.
**Method**: Analyzed 5 interlaced Caesar shifts. Recovered Key: **CVSUI**.

In [26]:
from collections import Counter

# --- CONFIGURAZIONE ---
ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ,.-"
MOD = len(ALPHABET)

# Frequenze Inglesi normalizzate (0.0 - 1.0)
# A-Z standard + stime per punteggiatura
ENGLISH_PROBS = {
    'E': 0.127, 'T': 0.091, 'A': 0.082, 'O': 0.075, 'I': 0.070, 'N': 0.067,
    'S': 0.063, 'H': 0.061, 'R': 0.060, 'D': 0.043, 'L': 0.040, 'U': 0.028,
    'C': 0.028, 'M': 0.024, 'W': 0.024, 'F': 0.022, 'Y': 0.020, 'G': 0.020,
    'P': 0.019, 'B': 0.015, 'V': 0.010, 'K': 0.008, 'J': 0.002, 'X': 0.002,
    'Q': 0.001, 'Z': 0.001,
    '.': 0.010, ',': 0.010, '-': 0.150 # Trattino come spazio
}

def chi_square_stat(text):
    """Calcola lo score Chi-Quadro rispetto all'inglese."""
    if not text: return float('inf')
    
    counts = Counter(text)
    length = len(text)
    score = 0
    
    for char in ALPHABET:
        observed = counts.get(char, 0)
        expected = length * ENGLISH_PROBS.get(char, 0)
        if expected > 0:
            score += ((observed - expected) ** 2) / expected
        elif observed > 0:
            # Penalità per caratteri molto rari o imprevisti se non sono nel dizionario
            score += 100 
            
    return score

# --- VIGENERE DISCOVERY: Statistical Approach ---
# Instead of guessing, we calculate the key mathematically.

def crack_vigenere_key(ciphertext, period):
    print(f"--- Automated Key Recovery (Period {period}) ---")
    recovered_key = ""
    
    for i in range(period):
        # Extract the i-th column
        col = ciphertext[i::period]
        best_char = '?'
        min_chi = float('inf')
        
        # Test all 29 candidates
        for char_idx in range(len(ALPHABET)):
            char = ALPHABET[char_idx]
            # Decrypt column with this char
            decrypted_col_chars = []
            shift = char_idx
            
            for c in col:
                if c in ALPHABET:
                    p_idx = (ALPHABET.index(c) - shift) % MOD
                    decrypted_col_chars.append(ALPHABET[p_idx])
            
            decrypted_col = "".join(decrypted_col_chars)
            
            # Calculate Chi-Squared score
            score = chi_square_stat(decrypted_col)
            
            if score < min_chi:
                min_chi = score
                best_char = char
        
        recovered_key += best_char
        print(f"Stream {i}: Best fit is '{best_char}' (Chi2: {min_chi:.2f})")
    
    return recovered_key

# Execution
# Assicurati che 'cipher3' sia caricato. Se dà errore, decommenta la riga sotto:
# cipher3 = read_ciphertext("177-Student/3.txt") 

if 'cipher3' in globals():
    key_vigenere = crack_vigenere_key(cipher3, period=5)
    print(f"\n✅ FOUND KEY: {key_vigenere}")
else:
    print("cipher3 not defined. Please run the file reading cell before this.")

--- Automated Key Recovery (Period 5) ---
Stream 0: Best fit is 'C' (Chi2: 64.36)
Stream 1: Best fit is 'V' (Chi2: 52.36)
Stream 2: Best fit is 'S' (Chi2: 50.54)
Stream 3: Best fit is 'U' (Chi2: 46.90)
Stream 4: Best fit is 'I' (Chi2: 54.94)

✅ FOUND KEY: CVSUI


In [9]:
cipher3 = read_ciphertext("3.txt")
KEY_VIGENERE = "CVSUI"

def vigenere_decrypt(text, key):
    res = []
    key_indices = [ALPHABET.index(k) for k in key]
    text_indices = [ALPHABET.index(c) for c in text]
    
    for i, t_idx in enumerate(text_indices):
        k_idx = key_indices[i % len(key)]
        p_idx = (t_idx - k_idx) % MOD
        res.append(ALPHABET[p_idx])
    return "".join(res)

plain3 = vigenere_decrypt(cipher3, KEY_VIGENERE)
print(f"--- Cipher 3 Decrypted (Key: {KEY_VIGENERE}) ---\n{plain3}")

--- Cipher 3 Decrypted (Key: CVSUI) ---
CANT.THINKOFTHESUNSETSANDTHEMOONRISES--IBELIEVETHECOLOURSARETHEREAREWILDPEACOCKS,RACHELHAZARDED.ANDMARVELLOUSCREATURESINTHEWATER,HELENASSERTED.ONEMIGHTDISCOVERANEWREPTILE,RACHELCONTINUED.THERESCERTAINTOBEAREVOLUTION,IMTOLD,HELENURGED.THEEFFECTOFTHESESUBTERFUGESWASALITTLEDASHEDBYRIDLEY,WHO,AFTERREGARDINGPEPPERFORSOMEMOMENTS,SIGHEDALOUD,POORFELLOWANDINWARDLYSPECULATEDUPONTHEUNKINDNESSOFWOMEN.HESTAYED,HOWEVER,INAPPARENTCONTENTMENTFORSIXDAYS,PLAYINGWITHAMICROSCOPEANDANOTEBOOKINONEOFTHEMANYSPARSELYFURNISHEDSITTING-ROOMS,BUTONTHEEVENINGOFTHESEVENTHDAY,ASTHEYSATATDINNER,HEAPPEAREDMORERESTLESSTHANUSUAL.THEDINNER-TABLEWASSETBETWEENTWOLONGWINDOWSWHICHWERELEFTUNCURTAINEDBYHELENSORDERS.DARKNESSFELLASSHARPLYASAKNIFEINTHISCLIMATE,ANDTHETOWNTHENSPRANGOUTINCIRCLESANDLINESOFBRIGHTDOTSBENEATHTHEM.BUILDINGSWHICHNEVERSHOWEDBYDAYSHOWEDBYNIGHT,ANDTHESEAFLOWEDRIGHTOVERTHELANDJUDGINGBYTHEMOVINGLIGHTSOFTHESTEAMERS.THESIGHTFULFILLEDTHESAMEPURPOSEASANORCHESTRAINALONDONREST